In [ ]:
# just some thinking about trying to implement computational neuroscience operations from the dayan and abbot book
# which should be really fascinating, and deriving correct algorithms for these things!


# may not be the most efficient representation, but having this as a type is probably worthwhile
mutable struct SpikeTrain
    spikes::Array{Spike}
end

mutable struct Spike
    time::Number
    val::Number
end

# do custom sorting via a custom comparator
sort!(s::SpikeTrain) = SpikeTrain(sort!(s.spikes, by=first))
sort!(s::Array{Spike}) = sort!(s, by=first)

# assumes sorted spiketrains which is absolutely required!
spike_count_rate(s::SpikeTrain) = length(s.spikes) / (last(s.spikes).time - first(s.spikes).time)

# the average number of spike counts observed over trials
spike_count_average(s::Array{SpikeTrain}) = mean([spike_count_rate(e) for e in s])

# compute simple bin averaging over spiketrains to get the time-dependent firing rate r(t)
function firing_rate_binned(s::SpikeTrain, num_bins::Int)
    dur = (last(s).time - first(s).time) / num_bins
    rates = []
    cur_bin_num::Int = 1
    spikes_per_bin = 0
    for spike in s
        t = spike.time
        if t > dur * cur_bin_num # check if flows over to the next bin. if not add it to the current bin
            push!(rates, spikes_per_bin / dur) # push the previous bin onto rates
            spikes_per_bin = 1
            cur_bin_num +=1
        else
            spikes_per_bin +=1
        end
    end
    # at the end push the final numbers
    push!(rates, spikes_per_bin)
    return rates
end
# do this the really inefficient double loop way. It'll be O(n^2). There is undoubtedly a better way
function firing_rate_window(s::SpikeTrain,window_func, window_size, window_resolution)
    step = (last(s).time - first(s).time) / window_resolution
    windows = []
    wind = 0
    for i in 1:window_resolution
        p = i * step
        for spike in s
            if s.time <= p + (window_size/2) && s.time >= p - (window_size/2) # I think this is the right conditional
                wind += window_func(s.time, p)
            end
        end
        push!(windows, wind)
    end
    return windows
end

# define some window functions!
rectangle(t,p) = 1 # always 1 as it just coutns the number in the window
causal_rectangle(t,p) = int(t < p) # only responds if the spike is less than the time of the point - i.e. each point can onl ylook into the past
gaussian(t,p,s) = exp(-(t-p)^2/2*s)
function causal_gaussian(t,p,s)
    if t < p
        return gaussian(t,p,s)
    end
    return 0
end
function causal_alpha(t,p,a)
    if t < p
        return a^2 * p * exp(- (a * p))
    end
    return 0
end
    
    
    
        
        
        
    

